In [1]:
import pandas as pd
import re
df=pd.read_excel(r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\19.Business Systems\Labour Hire Report\EXPORT_20250709041224.xlsx", header=0)

# Define mapping: keyword (case-insensitive) -> shift type
# Define keyword-based shift mapping (without 'nhd' for now)
keyword_map = {
    ' arvo ': 'Afternoon Shift',
    ' as we ': 'Afternoon Shift',
    '(arvo)': 'Afternoon Shift',
    ' ns ': 'Night Shift',
    ' nhs ': 'Night Shift',
    ' n/s ': 'Night Shift',
    ' o/t ': 'Overtime',
    ' 1.5': 'Overtime',
    ' @ 2.5 ': 'Overtime',
    ' @2.0': 'Overtime',
    ' et ': 'Overtime',
    ' x1.5 ': 'Overtime',
    ' t1.5 ': 'Overtime',
    ' ot1.5 ': 'Overtime',
    ' x2 ': 'Overtime',
    ' ot ': 'Overtime',
    ' d/t ': 'Overtime',
    ' dbl ': 'Overtime',
    ' ph ': 'Public Holiday',
    ' p/h ': 'Public Holiday',
    ' sat ': 'Weekend',
    ' sun ': 'Weekend',
    ' day ': 'Normal',
    'time and half':'Overtime',
    'o/time':'Overtime',
    'time half':'Overtime',
    'time and a half':'Overtime',
    'double time':'Overtime',
    ' ordinary ': 'Normal'
}

def classify_shift(description):
    desc = f" {str(description).lower()} "  # add padding to catch boundary matches
    
    # Special case: nhd + matraville = Normal
    if ' nhd ' in desc:
        if 'matraville' in desc:
            return 'Normal'
        else:
            return 'Night Shift'

    # General keyword scanning
    for keyword, shift in keyword_map.items():
        if keyword in desc:
            return shift

    return 'Normal'

# Apply logic
df['Shift Type'] = df['Material Description'].apply(classify_shift)


In [ ]:
#Plant Mapping
df_mapping = pd.read_excel(
    r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\19.Business Systems\Labour Hire Report\EXPORT_final.xlsx",
    sheet_name='Plant Name', header=1,
    usecols=[0, 1]
)

df=df.merge(df_mapping, on='Plant', how='left')

# Create new column using map
df['Plant Name'] = df['Description']

# Keep only the second 'Name 1' column and drop others
df = df.loc[:, ~df.columns.duplicated(keep='last')]
df = df[~(df.iloc[:, 0].isna() & df.iloc[:, 1].isna())]

# Return final columns in the df
df=df[['Material Description','Plant','Plant Name','Supplier','Name 1','Material Document','Posting Date','Amt.in Loc.Cur.','Qty in unit of entry','Reference','Purchase order','Cost Center','Shift Type']].rename(columns={'Qty in unit of entry':'Working Hours'})

with pd.ExcelWriter(
    r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\19.Business Systems\Labour Hire Report\EXPORT_final.xlsx",
    engine='openpyxl',
    mode='a',  # append mode
    if_sheet_exists='replace'  # replace the existing 'Labour Data' sheet
) as writer:
    df.to_excel(writer, sheet_name='Labour Data', index=False)